In [ ]:
import pandas as pd
import fiona
import geopandas as gpd
from fiona.crs import to_string

from madmex.models import Tag


# 7) Random sample para cada categoría generada de Kmeans

**La muestra debe generarse de modo que incluya los polígonos dados por inecol más otros y que en total sean 1000**

Intersección y contención del geopandas dataframe con el shapefile:

In [19]:
input_file='/LUSTRE/MADMEX/training/cultivos/igamma_veracruz/puntos_con_etiquetas/shp_puntos_martinez_lcc.shp'
input_file2='/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi_mean-ndvi_min-ndvi_max/02-01-2020/40_-31/02-01-2020_clusters_40_-31_nclusters_4.shp'

In [11]:
gdf_df_clusters = gpd.read_file(input_file2)

In [13]:
gdf_df_clusters.head()

,features_i,preds,geometry
0,537885,2,"POLYGON ((2977960.000 926120.000, 2977980.000 ..."
1,537675,2,"POLYGON ((2978250.000 926120.000, 2978420.000 ..."
2,537799,0,"POLYGON ((2978440.000 926120.000, 2978510.000 ..."
3,537676,2,"POLYGON ((2978610.000 926120.000, 2978650.000 ..."
4,537886,2,"POLYGON ((2979320.000 926120.000, 2979340.000 ..."


In [96]:
len(gdf_df_clusters.index)

208008

In [193]:
gdf_poi = gpd.read_file(input_file)

In [194]:
gdf_poi.head()

,CVE_MUN,Municip,DstrtDR,Ecoregn,ZonadVd,Name,Obsrvcn,geometry
0,124,Papantla,Martínez de la Torre,Veracruz moist forests,Bosque Subhúmedo premontano [Cálido - Lluvioso],Maíz,None,POINT (2987250.883 932708.313)
1,042,Colipa,Martínez de la Torre,Veracruz moist forests,Bosque Húmedo premontano [Cálido - Lluvioso],Maíz,None,POINT (3050582.748 894254.131)
2,023,Atzalan,Martínez de la Torre,Veracruz moist forests,Bosque Lluvioso premontano [Cálido - Muy Lluvi...,Pastizal,None,POINT (3007166.751 884499.105)
3,023,Atzalan,Martínez de la Torre,Veracruz moist forests,Bosque Lluvioso premontano [Cálido - Muy Lluvi...,Plátano,IZQ,POINT (3004236.090 883061.625)
4,197,Yecuatla,Martínez de la Torre,Oaxacan montane forests,Bosque Húmedo montano [Templado - Lluvioso],Maíz,IZQ,POINT (3040558.914 879107.389)


**Interesan los id's que intersectan o contienen los puntos pues se extraerán tales id's del gdf_df_clusters y a los restantes se les extraerá una muestra**

Obtener `features_id` de `gdf_df_clusters`  que intersectan o contienen `gdf_poi`

In [288]:
gdf_df_clusters_contains_poi = gpd.sjoin(gdf_df_clusters,gdf_poi,how='inner',op='contains').drop_duplicates(subset='features_i')

In [289]:
gdf_df_clusters_contains_poi.head()

,features_i,preds,geometry,index_right,CVE_MUN,Municip,DstrtDR,Ecoregn,ZonadVd,Name,Obsrvcn
1280,534719,0,"POLYGON ((2984820.000 925930.000, 2984880.000 ...",566,124,Papantla,Martínez de la Torre,Veracruz moist forests,Bosque Subhúmedo premontano [Cálido - Lluvioso],Maíz,IZQ ATRAS PAPAYA
1301,534675,3,"POLYGON ((3014490.000 926050.000, 3014490.000 ...",1293,102,Martínez de la Torre,Martínez de la Torre,Veracruz moist forests,Bosque Subhúmedo premontano [Cálido - Lluvioso],Pastizal,LADO
1865,533540,3,"POLYGON ((2982510.000 926030.000, 2982540.000 ...",759,124,Papantla,Martínez de la Torre,Veracruz moist forests,Bosque Subhúmedo premontano [Cálido - Lluvioso],Maíz,None
2142,533080,0,"POLYGON ((3015370.000 925730.000, 3015390.000 ...",1273,102,Martínez de la Torre,Martínez de la Torre,Veracruz moist forests,Bosque Subhúmedo premontano [Cálido - Lluvioso],Pastizal,IZQ
2900,531041,0,"POLYGON ((3016300.000 925690.000, 3016350.000 ...",278,102,Martínez de la Torre,Martínez de la Torre,Veracruz moist forests,Bosque Subhúmedo premontano [Cálido - Lluvioso],Cítrico,NARANJA


In [291]:
len(gdf_df_clusters_contains_poi['features_i'])

647

In [292]:
gdf_df_clusters_poi_intersects = gpd.sjoin(gdf_df_clusters,gdf_poi,op='intersects').drop_duplicates(subset='features_i')

In [293]:
gdf_df_clusters_poi_intersects.head()

,features_i,preds,geometry,index_right,CVE_MUN,Municip,DstrtDR,Ecoregn,ZonadVd,Name,Obsrvcn
1280,534719,0,"POLYGON ((2984820.000 925930.000, 2984880.000 ...",566,124,Papantla,Martínez de la Torre,Veracruz moist forests,Bosque Subhúmedo premontano [Cálido - Lluvioso],Maíz,IZQ ATRAS PAPAYA
1301,534675,3,"POLYGON ((3014490.000 926050.000, 3014490.000 ...",1293,102,Martínez de la Torre,Martínez de la Torre,Veracruz moist forests,Bosque Subhúmedo premontano [Cálido - Lluvioso],Pastizal,LADO
1865,533540,3,"POLYGON ((2982510.000 926030.000, 2982540.000 ...",759,124,Papantla,Martínez de la Torre,Veracruz moist forests,Bosque Subhúmedo premontano [Cálido - Lluvioso],Maíz,None
2142,533080,0,"POLYGON ((3015370.000 925730.000, 3015390.000 ...",1273,102,Martínez de la Torre,Martínez de la Torre,Veracruz moist forests,Bosque Subhúmedo premontano [Cálido - Lluvioso],Pastizal,IZQ
2900,531041,0,"POLYGON ((3016300.000 925690.000, 3016350.000 ...",278,102,Martínez de la Torre,Martínez de la Torre,Veracruz moist forests,Bosque Subhúmedo premontano [Cálido - Lluvioso],Cítrico,NARANJA


In [294]:
len(gdf_df_clusters_poi_intersects.index)

647

In [295]:
t_intersects_rows = tuple(gdf_df_clusters_poi_intersects['features_i'])

In [365]:
gdf_df_clusters_contains_poi_reduced = gpd.GeoDataFrame()

In [366]:
gdf_df_clusters_contains_poi_reduced['features_i'] = gdf_df_clusters_contains_poi['features_i']
gdf_df_clusters_contains_poi_reduced['preds'] = gdf_df_clusters_contains_poi['preds']
gdf_df_clusters_contains_poi_reduced['geometry'] = gdf_df_clusters_contains_poi['geometry']

In [367]:
gdf_df_clusters_contains_poi_reduced.head()

,features_i,preds,geometry
1280,534719,0,"POLYGON ((2984820.000 925930.000, 2984880.000 ..."
1301,534675,3,"POLYGON ((3014490.000 926050.000, 3014490.000 ..."
1865,533540,3,"POLYGON ((2982510.000 926030.000, 2982540.000 ..."
2142,533080,0,"POLYGON ((3015370.000 925730.000, 3015390.000 ..."
2900,531041,0,"POLYGON ((3016300.000 925690.000, 3016350.000 ..."


In [368]:
len(gdf_df_clusters_contains_poi_reduced.index)

647

In [369]:
sum(gdf_df_clusters_contains_poi_reduced['features_i'].isin(t_intersects_rows))

647

In [370]:
gdf_df_clusters_keep = gdf_df_clusters_contains_poi_reduced.loc[gdf_df_clusters_contains_poi_reduced['features_i'].isin(t_intersects_rows)]

In [371]:
gdf_df_clusters_keep.head()

,features_i,preds,geometry
1280,534719,0,"POLYGON ((2984820.000 925930.000, 2984880.000 ..."
1301,534675,3,"POLYGON ((3014490.000 926050.000, 3014490.000 ..."
1865,533540,3,"POLYGON ((2982510.000 926030.000, 2982540.000 ..."
2142,533080,0,"POLYGON ((3015370.000 925730.000, 3015390.000 ..."
2900,531041,0,"POLYGON ((3016300.000 925690.000, 3016350.000 ..."


In [372]:
len(gdf_df_clusters_keep.index)

647

**If number of rows are less than 1000 then filter_out rows that don't contain nor intersect points and then sample**

In [353]:
t_keep_rows = tuple(gdf_df_clusters_keep['features_i'])

In [354]:
len(t_keep_rows)

647

In [355]:
t_keep_rows[0:5]

(534719, 534675, 533540, 533080, 531041)

In [315]:
gdf_df_clusters_to_sample = gdf_df_clusters.loc[~gdf_df_clusters['features_i'].isin(t_keep_rows)]

In [316]:
gdf_df_clusters_to_sample.head()

,features_i,preds,geometry
0,537885,2,"POLYGON ((2977960.000 926120.000, 2977980.000 ..."
1,537675,2,"POLYGON ((2978250.000 926120.000, 2978420.000 ..."
2,537799,0,"POLYGON ((2978440.000 926120.000, 2978510.000 ..."
3,537676,2,"POLYGON ((2978610.000 926120.000, 2978650.000 ..."
4,537886,2,"POLYGON ((2979320.000 926120.000, 2979340.000 ..."


In [317]:
len(gdf_df_clusters_to_sample.index)

207361

**Extraer muestra**

Using https://stackoverflow.com/questions/41345289/getting-a-random-sample-in-python-dataframe-by-category

In [319]:
gdf_df_clusters_to_sample.head()

,features_i,preds,geometry
0,537885,2,"POLYGON ((2977960.000 926120.000, 2977980.000 ..."
1,537675,2,"POLYGON ((2978250.000 926120.000, 2978420.000 ..."
2,537799,0,"POLYGON ((2978440.000 926120.000, 2978510.000 ..."
3,537676,2,"POLYGON ((2978610.000 926120.000, 2978650.000 ..."
4,537886,2,"POLYGON ((2979320.000 926120.000, 2979340.000 ..."


In [337]:
nmultipolygons=len(gdf_df_clusters_to_sample.index)

In [338]:
nmultipolygons

207361

In [339]:
total_mpolygons = 1000
nsample=total_mpolygons-len(t_keep_rows)

In [340]:
nsample

353

In [341]:
nclusters=4

In [342]:
nclusters

4

In [343]:
if nmultipolygons>nsample:
    nsample_per_cluster=int(nsample/nclusters)
    nsample=nsample_per_cluster*nclusters
else:
    nsample_per_cluster=int(nmultipolygons/nclusters)
    nsample=nsample_per_cluster*nclusters

In [344]:
(nmultipolygons,nsample_per_cluster,nsample_per_cluster*nclusters,nsample)

(207361, 88, 352, 352)

In [345]:
gdf_df_clusters_sample=gdf_df_clusters_to_sample.groupby('preds').apply(lambda s: s.sample(min(len(s), nsample_per_cluster)))

In [346]:
gdf_df_clusters_sample.head()

features_i  preds  \
preds                             
0     68669       359363      0   
      50765       397571      0   
      192104       57469      0   
      122774      250605      0   
      142348      197674      0   

                                                       geometry  
preds                                                            
0     68669   POLYGON ((3019000.000 910770.000, 3019010.000 ...  
      50765   POLYGON ((3027290.000 913960.000, 3027300.000 ...  
      192104  POLYGON ((3013610.000 881110.000, 3013630.000 ...  
      122774  POLYGON ((3004540.000 901160.000, 3004550.000 ...  
      142348  POLYGON ((3025890.000 896110.000, 3025930.000 ...

In [347]:
gdf_df_clusters_sample.reset_index(drop=True,inplace=True)

In [348]:
gdf_df_clusters_sample.head()

,features_i,preds,geometry
0,359363,0,"POLYGON ((3019000.000 910770.000, 3019010.000 ..."
1,397571,0,"POLYGON ((3027290.000 913960.000, 3027300.000 ..."
2,57469,0,"POLYGON ((3013610.000 881110.000, 3013630.000 ..."
3,250605,0,"POLYGON ((3004540.000 901160.000, 3004550.000 ..."
4,197674,0,"POLYGON ((3025890.000 896110.000, 3025930.000 ..."


In [349]:
len(gdf_df_clusters_sample.index)

352

**Union of sample and keep polygons**

In [415]:
gdf_sampled_keep = pd.concat([gdf_df_clusters_sample,gdf_df_clusters_keep],ignore_index=True,sort=False).pipe(gpd.GeoDataFrame)

In [416]:
gdf_sampled_keep.head()

,features_i,preds,geometry
0,359363,0,"POLYGON ((3019000.000 910770.000, 3019010.000 ..."
1,397571,0,"POLYGON ((3027290.000 913960.000, 3027300.000 ..."
2,57469,0,"POLYGON ((3013610.000 881110.000, 3013630.000 ..."
3,250605,0,"POLYGON ((3004540.000 901160.000, 3004550.000 ..."
4,197674,0,"POLYGON ((3025890.000 896110.000, 3025930.000 ..."


In [417]:
len(gdf_sampled_keep.index)

999

# 8) Write results to file

In [3]:
date_today = '02-01-2020'
dc_tile = '40_-31'
path_result_dc_tile = '/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi_mean-ndvi_min-ndvi_max/02-01-2020/40_-31/'
features_string = 'ndvi_mean-ndvi_min-ndvi_max'

**Geopackage**

In [379]:
path_results_geopackage = '/home/madmex_user/agriculture_geopackages/features_computed/' + features_string + '/'

In [380]:
path_results_geopackage

'/home/madmex_user/agriculture_geopackages/features_computed/ndvi_mean-ndvi_min-ndvi_max/'

In [383]:
path_result_geopackage = os.path.join(path_results_geopackage, date_today 
                                       + '/' + dc_tile + '/')

In [384]:
path_result_geopackage

'/home/madmex_user/agriculture_geopackages/features_computed/ndvi_mean-ndvi_min-ndvi_max/02-01-2020/40_-31/'

In [385]:
if not os.path.exists(path_result_geopackage):
    os.makedirs(path_result_geopackage)

In [387]:
clusters_filename = path_result_geopackage + date_today + '_clusters_' +  dc_tile + '_nclusters_%d' % nclusters + '_sample_%d' % total_mpolygons + '.gpkg'

In [388]:
clusters_filename

'/home/madmex_user/agriculture_geopackages/features_computed/ndvi_mean-ndvi_min-ndvi_max/02-01-2020/40_-31/02-01-2020_clusters_40_-31_nclusters_4_sample_1000.gpkg'

In [389]:
layer_name = date_today + '_clusters_' +  dc_tile + '_nclusters_%d' % nclusters + '_sample_%d' % total_mpolygons

In [390]:
layer_name

'02-01-2020_clusters_40_-31_nclusters_4_sample_1000'

In [393]:
gdf_sampled_keep.crs = gdf_poi.crs

In [394]:
gdf_sampled_keep.crs

{'proj': 'lcc',
 'lat_1': 17.5,
 'lat_2': 29.5,
 'lat_0': 12,
 'lon_0': -102,
 'x_0': 2500000,
 'y_0': 0,
 'datum': 'WGS84',
 'units': 'm',
 'no_defs': True}

In [395]:
start_time=time.time()

gdf_sampled_keep.to_file(clusters_filename,
                         layer=layer_name,
                         driver='GPKG')
end_time=time.time()

In [396]:
secs = end_time-start_time
secs

0.3745229244232178

In [399]:
shutil.move(clusters_filename,path_result_dc_tile)

'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi_mean-ndvi_min-ndvi_max/02-01-2020/40_-31/02-01-2020_clusters_40_-31_nclusters_4_sample_1000.gpkg'

**Shapefile**

In [407]:
clusters_filename = path_result_dc_tile + date_today + '_clusters_' +  dc_tile + '_nclusters_%d' % nclusters + '_sample_%d' % total_mpolygons + '.shp'

In [409]:
clusters_filename

'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi_mean-ndvi_min-ndvi_max/02-01-2020/40_-31/02-01-2020_clusters_40_-31_nclusters_4_sample_1000.shp'

In [403]:
gdf_sampled_keep.crs = gdf_poi.crs

In [404]:
gdf_sampled_keep.crs

{'proj': 'lcc',
 'lat_1': 17.5,
 'lat_2': 29.5,
 'lat_0': 12,
 'lon_0': -102,
 'x_0': 2500000,
 'y_0': 0,
 'datum': 'WGS84',
 'units': 'm',
 'no_defs': True,
 'wktext': True}

In [410]:
start_time=time.time()
gdf_sampled_keep.to_file(clusters_filename)
end_time=time.time()

In [411]:
secs = end_time-start_time
secs

0.3954622745513916

# 9) Asignar etiquetas numéricas al shapefile dado que trae etiquetas

In [5]:
path_shapefile_with_classes = '/LUSTRE/MADMEX/training/cultivos/igamma_veracruz/puntos_con_etiquetas/'

In [6]:
input_file = path_shapefile_with_classes + 'shp_puntos_martinez_lcc.shp'

In [7]:
input_file

'/LUSTRE/MADMEX/training/cultivos/igamma_veracruz/puntos_con_etiquetas/shp_puntos_martinez_lcc.shp'

In [520]:
with fiona.open(input_file) as src:
    fc=list(src)
    crs = to_string(src.crs)

In [497]:
fc[0]

{'type': 'Feature',
 'id': '0',
 'properties': OrderedDict([('CVE_MUN', '124'),
              ('Municip', 'Papantla'),
              ('DstrtDR', 'Martínez de la Torre'),
              ('Ecoregn', 'Veracruz moist forests'),
              ('ZonadVd', 'Bosque Subhúmedo premontano [Cálido - Lluvioso]'),
              ('Name', 'Maíz'),
              ('Obsrvcn', None)]),
 'geometry': {'type': 'Point',
  'coordinates': (2987250.8825278645, 932708.3126494994)}}

In [498]:
name_attribute='Name'

In [535]:
fc_subset = [{'id': feature['id'],
              'name': feature['properties'][name_attribute],
              'geometry': feature['geometry']} for feature in fc]

In [536]:
fc_subset[0]

{'id': '0',
 'name': 'Maíz',
 'geometry': {'type': 'Point',
  'coordinates': (2987250.8825278645, 932708.3126494994)}}

In [537]:
def normalize_name_classes(string):
    return unicodedata.normalize('NFKD', string).encode('ASCII','ignore').decode('utf-8').lower()

In [538]:
for k in range(len(fc_subset)):
    fc_subset[k]['name'] =  normalize_name_classes(fc_subset[k]['name'])

In [539]:
fc_subset[0]

{'id': '0',
 'name': 'maiz',
 'geometry': {'type': 'Point',
  'coordinates': (2987250.8825278645, 932708.3126494994)}}

In [540]:
classes = list(set([feature['name'] for feature in fc_subset]))

In [541]:
classes

['pastizal', 'maiz', 'mango', 'platano', 'cafe', 'citrico', 'cafe sol']

In [542]:
len(classes)

7

In [543]:
def add_numeric_code_to_classes(classes):
    return {classes[k]: k for k in range(len(classes))}

In [544]:
classes = add_numeric_code_to_classes(classes)

In [545]:
classes

{'pastizal': 0,
 'maiz': 1,
 'mango': 2,
 'platano': 3,
 'cafe': 4,
 'citrico': 5,
 'cafe sol': 6}

# 10) Insertar en madmex_Tag y añadir numeric_code a fc_subset

**Verificar credenciales de la DB a la que se insertará**

In [577]:
scheme_tag = 'agricultura_completo'

In [473]:
[Tag.objects.get_or_create(scheme=scheme_tag, value=name_class, numeric_code=classes[name_class],color='') for name_class in classes]

[(<Tag: Tag object (32)>, False),
 (<Tag: Tag object (33)>, False),
 (<Tag: Tag object (34)>, False),
 (<Tag: Tag object (35)>, False),
 (<Tag: Tag object (36)>, False),
 (<Tag: Tag object (37)>, False),
 (<Tag: Tag object (38)>, False)]

In [546]:
def get_numeric_code(string):
    return classes[string]

In [547]:
fc_subset = [{'id': feature['id'],
               'name': feature['name'],
               'code': get_numeric_code(feature['name']),
               'geometry': feature['geometry']} for feature in fc_subset]

In [548]:
fc_subset[10]

{'id': '10',
 'name': 'cafe sol',
 'code': 6,
 'geometry': {'type': 'Point',
  'coordinates': (3013642.151646747, 880227.5095511458)}}

In [549]:
fc_subset_schema = {'geometry': 'Point',
                    'properties': {'name': 'str',
                                   'code': 'int'}
                   }

In [550]:
output_file_with_numeric_codes = path_shapefile_with_classes + 'shp_puntos_martinez_with_numeric_code_lcc.shp'

In [551]:
output_file_with_numeric_codes

'/LUSTRE/MADMEX/training/cultivos/igamma_veracruz/puntos_con_etiquetas/shp_puntos_martinez_with_numeric_code_lcc.shp'

In [552]:
driver='ESRI Shapefile'

In [553]:
layer='shp_puntos_martinez_with_numeric_code_lcc'

In [556]:
with fiona.open(output_file_with_numeric_codes, 'w',
                encoding='utf-8',
                driver=driver,
                layer=layer,
                crs=crs,
                schema=fc_subset_schema) as dst:
    [dst.write({'geometry': feature['geometry'],
                'properties': {'name': feature['name'],
                               'code': feature['code']}
                }) for feature in fc_subset]

# 11) Registrar cultivos con etiquetas numéricas en trainclassification con esquema "agriculture_completo"

In [3]:
file_training = '/LUSTRE/MADMEX/training/cultivos/igamma_veracruz/puntos_con_etiquetas/shp_puntos_martinez_with_numeric_code_lcc.shp'

In [5]:
%%bash
/home/madmex_user/.local/bin/antares ingest_training_from_vector /LUSTRE/MADMEX/training/cultivos/igamma_veracruz/puntos_con_etiquetas/shp_puntos_martinez_with_numeric_code_lcc.shp --scheme agricultura_completo --year 2020 --name train_cultivos_inecol --field code

Command line arguments: {'verbosity': 1, 'settings': None, 'pythonpath': None, 'traceback': False, 'no_color': False, 'force_color': False, 'input_file': '/LUSTRE/MADMEX/training/cultivos/igamma_veracruz/puntos_con_etiquetas/shp_puntos_martinez_with_numeric_code_lcc.shp', 'scheme': 'agricultura_completo', 'field': 'code', 'name': 'train_cultivos_inecol', 'year': '2020', 'app': False}
Command execution is done in 15.899461030960083 seconds.
2020-02-05 19:40:43,192 - madmex.management.base - base execute: Command execution is done in 15.899461030960083 seconds.


# 12) Registrar segmentos con etiquetas automáticas en trainclassificationlabeledbyapp con esquema "automatic_4_classes"

In [413]:
%%bash
/home/madmex_user/.local/bin/antares ingest_training_from_vector \
/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi_mean-ndvi_min-ndvi_max/02-01-2020/40_-31/02-01-2020_clusters_40_-31_nclusters_4_sample_1000.shp \
--scheme automatic_4_classes --year 2020 --name agricultura_4_clases_sample_1000 --field preds --app

Command line arguments: {'verbosity': 1, 'settings': None, 'pythonpath': None, 'traceback': False, 'no_color': False, 'force_color': False, 'input_file': '/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi_mean-ndvi_min-ndvi_max/02-01-2020/40_-31/02-01-2020_clusters_40_-31_nclusters_4_sample_1000.shp', 'scheme': 'automatic_4_classes', 'field': 'preds', 'name': 'agricultura_4_clases_sample_1000', 'year': '2020', 'app': True}
Command execution is done in 12.692714214324951 seconds.
2020-02-06 17:36:11,600 - madmex.management.base - base execute: Command execution is done in 12.692714214324951 seconds.


# 13) Asignar etiquetas numéricas a los clústers de segmentos que intersectan con shapefile de cultivos